In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/prop-1.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0      prop-1        4    2    4    0    2    5     1   0   2  ...  1.000000   
1      prop-1        4   12    4    1   19   44    66   2  18  ...  0.000000   
2      prop-1        4   10    4    0   13   48    45   4  13  ...  1.000000   
3      prop-1        4    2    3    0    5    7     1   0   5  ...  1.000000   
4      prop-1        4    8    1    0    4   38     0   3   1  ...  0.500000   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
18466  prop-1      185   17    1    0   24   67    96   9  15  ...  0.150000   
18467  prop-1      185    9    1    0   25   49    14   7  18  ...  0.333333   
18468  prop-1      185    6    3    0   10   29     0   0  10  ...  0.800000   
18469  prop-1      185    5    5    0   28    9    10  26   2  ...  1.000000   
18470  prop-1      185   11    1    0   54   34     9  51   3  ...  0.666667   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0        0  0.941176  0.750000   1    1  10.500000       3  1.5000    0  
1        0  0.905172  0.350000   2    3  20.583333       8  2.0833    0  
2        0  0.918919  0.500000   2    3  19.100000       4  1.2000    0  
3        0  0.933333  0.750000   1    1  11.500000       4  2.0000    1  
4        0  0.000000  0.285714   0    0  30.625000       2  0.8750    0  
...    ...       ...       ...  ..  ...        ...     ...     ...  ...  
18466    0  0.000000  0.250000   0    0  41.529412       6  2.6471    1  
18467    0  0.000000  0.319444   0    0  29.888889       3  1.3333    0  
18468    0  0.818182  0.400000   0    0  45.500000       6  2.1667    0  
18469    0  0.962963  0.650000   1    1   4.400000       1  0.2000    0  
18470    1  0.000000  0.400000   0    0  25.545455       6  1.9091    0  

[18471 rows x 23 columns]

In [4]:
ant_master = ant_master.loc[ant_master['version'] == 85]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
7075   prop-1       85    4    3    0   11   12     0   3   8  ...  1.000000   
7076   prop-1       85    0    1    0    0    0     0   0   0  ...  0.000000   
7077   prop-1       85    2    2    0    3    4     1   0   3  ...  0.000000   
7078   prop-1       85   55    3   10   72  218  1263  23  57  ...  0.217391   
7079   prop-1       85    2    1    0   10   25     0   1   9  ...  1.000000   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
10147  prop-1       85    5    5    0    3    9    10   1   2  ...  1.000000   
10148  prop-1       85    5    5    0    9    9    10   7   2  ...  1.000000   
10149  prop-1       85    6    3    0   27   50     3   0  27  ...  0.625000   
10150  prop-1       85    5    1    0   12   25     8   1  11  ...  0.500000   
10151  prop-1       85    2    3    0    8   20     1   0   8  ...  0.000000   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
7075     1  0.812500  0.500000   0    0  23.000000       1  0.7500    2  
7076     0  0.000000  0.000000   0    0   0.000000       0  0.0000    0  
7077     0  0.941176  0.666667   1    1   4.000000       1  0.5000    0  
7078     2  0.442105  0.160494   1    5  26.854545      25  2.2727    8  
7079     0  0.000000  0.666667   0    0  42.000000       5  2.5000    0  
...    ...       ...       ...  ..  ...        ...     ...     ...  ...  
10147    0  0.962963  0.650000   1    1   4.400000       1  0.2000    0  
10148    0  0.962963  0.650000   1    1   4.400000       1  0.2000    0  
10149    1  0.807692  0.416667   0    0  57.166667       4  2.0000    1  
10150    0  0.000000  0.750000   0    0  16.200000       2  0.8000    0  
10151    0  0.875000  0.666667   1    1  46.000000       5  2.5000    1  

[3077 rows x 23 columns]

In [5]:
train, test = train_test_split(ant_master, test_size=0.2)

In [6]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [7]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc       dam  \
7391     2    3    0    7   17     1   0   7    1  2.000000   96  1.000000   
9376     3    1    0    9   16     1   2   7    1  0.875000   88  0.250000   
9628     9    3    0   12   49    20   1  11    2  0.980769  390  0.923077   
9772     4    1    0    4    6     6   1   3    4  2.000000   13  0.000000   
10140    9    1    0   18   51    34   3  15    7  0.937500  287  0.000000   
...    ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...       ...   
7993     2    5    0    5   13     1   1   4    1  2.000000   66  0.000000   
9973     2    3    0    5    6     1   0   5    2  2.000000   13  0.000000   
9269     9    1    0    5   44    30   4   3    5  0.781250  332  0.750000   
9209     1    1    0    0    2     0   0   0    1  2.000000   32  0.000000   
8433     3    3    0    8   15     1   0   8    2  0.750000   50  0.500000   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  
7391     0  0.933333  0.750000   1    1  46.500000       9  4.5000  
9376     0  0.000000  0.500000   0    0  27.000000       5  1.6667  
9628     2  0.774194  0.625000   1    3  40.888889      11  3.4444  
9772     0  0.000000  0.562500   0    0   2.250000       1  0.7500  
10140    0  0.000000  0.142857   0    0  30.666667       4  2.6667  
...    ...       ...       ...  ..  ...        ...     ...     ...  
7993     0  0.990099  1.000000   2    2  32.000000       1  0.5000  
9973     0  0.875000  0.666667   1    1   5.500000       1  0.5000  
9269     1  0.000000  0.458333   0    0  35.444444       2  1.0000  
9209     0  0.000000  1.000000   0    0   3.000000       0  0.0000  
8433     0  0.875000  0.666667   1    1  15.000000       3  1.0000  

[2461 rows x 20 columns]

In [8]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc  dam  moa  \
7871     1    1    0    0    5     0   0   0    0  2.000000   17  0.0    0   
9070     1    1    0    0    2     0   0   0    1  2.000000    5  0.0    0   
7594     3    1    0    7   12     3   0   7    3  2.000000   33  0.0    0   
8218     2    3    0    3    5     1   0   3    1  2.000000   20  1.0    0   
10053   14    1    0   47  117    57   4  43   10  0.938462  717  0.3    1   
...    ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...  ...  ...   
8198     2    1    0    9   17     1   0   9    2  2.000000   44  0.0    0   
7363     2    4    0    1    3     1   0   1    1  2.000000    7  0.0    0   
8329     2    2    0    3    3     1   1   2    1  2.000000   12  0.0    0   
8578     3    1    0    2   11     3   2   0    2  2.000000   89  0.0    0   
7962     2    4    0   11   24     1   0  11    2  1.000000  216  0.0    0   

            mfa       cam  ic  cbm         amc  max_cc  avg_cc  
7871   0.000000  0.000000   0    0   12.000000       0  0.0000  
9070   0.000000  1.000000   0    0    3.000000       0  0.0000  
7594   0.000000  0.777778   0    0   10.000000       1  0.6667  
8218   0.933333  0.750000   1    1    8.500000       3  1.5000  
10053  0.000000  0.219780   0    0   49.500000      19  2.7143  
...         ...       ...  ..  ...         ...     ...     ...  
8198   0.000000  0.666667   0    0   21.000000       1  0.5000  
7363   0.888889  1.000000   1    2    2.500000       1  0.5000  
8329   0.666667  0.750000   1    1    5.000000       2  1.0000  
8578   0.000000  0.500000   0    0   28.666667       5  3.3333  
7962   0.980000  1.000000   2    6  105.500000      15  7.5000  

[616 rows x 20 columns]

In [9]:
y_train = train['bug']
y_test = test['bug']

In [10]:
y_train

7391     0
9376     0
9628     0
9772     0
10140    1
        ..
7993     0
9973     0
9269     0
9209     0
8433     1
Name: bug, Length: 2461, dtype: int64

In [11]:
y_test

7871     1
9070     0
7594     0
8218     0
10053    3
        ..
8198     0
7363     0
8329     0
8578     0
7962     0
Name: bug, Length: 616, dtype: int64

In [12]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [13]:
x_train_np

array([[ 2.        ,  3.        ,  0.        , ..., 46.5       ,
         9.        ,  4.5       ],
       [ 3.        ,  1.        ,  0.        , ..., 27.        ,
         5.        ,  1.6667    ],
       [ 9.        ,  3.        ,  0.        , ..., 40.88888889,
        11.        ,  3.4444    ],
       ...,
       [ 9.        ,  1.        ,  0.        , ..., 35.44444444,
         2.        ,  1.        ],
       [ 1.        ,  1.        ,  0.        , ...,  3.        ,
         0.        ,  0.        ],
       [ 3.        ,  3.        ,  0.        , ..., 15.        ,
         3.        ,  1.        ]])

In [14]:
x_test_np

array([[  1.        ,   1.        ,   0.        , ...,  12.        ,
          0.        ,   0.        ],
       [  1.        ,   1.        ,   0.        , ...,   3.        ,
          0.        ,   0.        ],
       [  3.        ,   1.        ,   0.        , ...,  10.        ,
          1.        ,   0.6667    ],
       ...,
       [  2.        ,   2.        ,   0.        , ...,   5.        ,
          2.        ,   1.        ],
       [  3.        ,   1.        ,   0.        , ...,  28.66666667,
          5.        ,   3.3333    ],
       [  2.        ,   4.        ,   0.        , ..., 105.5       ,
         15.        ,   7.5       ]])

In [15]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [16]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [17]:
train_x

array([[[ 2.        ,  3.        ,  0.        , ..., 46.5       ,
          9.        ,  4.5       ]],

       [[ 3.        ,  1.        ,  0.        , ..., 27.        ,
          5.        ,  1.6667    ]],

       [[ 9.        ,  3.        ,  0.        , ..., 40.88888889,
         11.        ,  3.4444    ]],

       ...,

       [[ 9.        ,  1.        ,  0.        , ..., 35.44444444,
          2.        ,  1.        ]],

       [[ 1.        ,  1.        ,  0.        , ...,  3.        ,
          0.        ,  0.        ]],

       [[ 3.        ,  3.        ,  0.        , ..., 15.        ,
          3.        ,  1.        ]]])

In [18]:
test_y

array([ 1,  0,  0,  0,  3,  1,  0,  0,  0,  1,  0,  0,  0,  1,  0,  0,  0,
        1,  0,  0,  0,  0,  2,  0,  2,  0,  2,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  1,  0,  0,  0, 12,  0,  0,  1,  0,  0,  0,  1,  0,  0,
        2,  0,  0,  1,  5,  0,  1,  0,  0,  0,  0,  0,  1,  1,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  1,  0,  1,  0,  0,  0,
        0,  0,  1,  0,  0,  0,  1,  0,  4,  0,  0,  0,  2,  7,  0,  1, 13,
        0,  0,  0,  1,  1,  1,  0,  1,  0,  1,  0,  0,  0,  1,  0,  0,  0,
        0,  1,  0,  0,  1,  5,  0,  0,  0,  0,  0,  0,  0,  1,  0,  3,  7,
        0,  0,  0,  0,  1,  1,  0,  0,  2,  0,  1,  1,  0,  1,  2,  0,  1,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  3,  0,  1,  0,  1,  0,  0,
        0,  2, 17,  5,  1,  1,  1,  3,  0,  0,  2,  5,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  1,  0,  4,  0,  0,  2,  0,  0,  1,  0,  1,  0,
        1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0,  1,  0,  0,  1,
        0,  5,  0,  0,  0

In [19]:
def define_model():
    # Designing and initializing the model.
    model = Sequential()
    model.add(LSTM(200, return_sequences=True))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(50, dropout = 0.2, return_sequences=True))
    model.add(LSTM(20, return_sequences=False))
    model.add(Dense(1, activation = 'relu'))
    model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0005) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )
    return model

In [20]:
y = list(test_y)

In [21]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(10):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/propv85/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(train_x, train_y, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(test_x, test_y)
    total_scores.append(score) 
    predictions = model.predict(test_x)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
25/25 [==============================] - 4s 12ms/step - loss: 5.2023 - mse: 5.2023 - mae: 0.8233 - root_mean_squared_error: 2.2808 - mean_squared_logarithmic_error: 0.3992
Epoch 2/100
25/25 [==============================] - 0s 6ms/step - loss: 4.7296 - mse: 4.7296 - mae: 1.0238 - root_mean_squared_error: 2.1748 - mean_squared_logarithmic_error: 0.3745
Epoch 3/100
25/25 [==============================] - 0s 6ms/step - loss: 4.6138 - mse: 4.6138 - mae: 1.0652 - root_mean_squared_error: 2.1480 - mean_squared_logarithmic_error: 0.3889
Epoch 4/100
25/25 [==============================] - 0s 7ms/step - loss: 4.4973 - mse: 4.4973 - mae: 1.0147 - root_mean_squared_error: 2.1207 - mean_squared_logarithmic_error: 0.3537
Epoch 5/100
25/25 [==============================] - 0s 6ms/step - loss: 4.2957 - mse: 4.2957 - mae: 0.9718 - root_mean_squared_error: 2.0726 - mean_squared_logarithmic_error: 0.3255
Epoch 6/100
25/25 [==============================] - 0s 6ms/step - loss: 4

25/25 [==============================] - 0s 6ms/step - loss: 2.7250 - mse: 2.7250 - mae: 0.7687 - root_mean_squared_error: 1.6508 - mean_squared_logarithmic_error: 0.2244
Epoch 89/100
25/25 [==============================] - 0s 6ms/step - loss: 2.5766 - mse: 2.5766 - mae: 0.7256 - root_mean_squared_error: 1.6052 - mean_squared_logarithmic_error: 0.2061
Epoch 90/100
25/25 [==============================] - 0s 6ms/step - loss: 2.8821 - mse: 2.8821 - mae: 0.7885 - root_mean_squared_error: 1.6977 - mean_squared_logarithmic_error: 0.2346
Epoch 91/100
25/25 [==============================] - 0s 6ms/step - loss: 2.4857 - mse: 2.4857 - mae: 0.7359 - root_mean_squared_error: 1.5766 - mean_squared_logarithmic_error: 0.2104
Epoch 92/100
25/25 [==============================] - 0s 6ms/step - loss: 2.3558 - mse: 2.3558 - mae: 0.7236 - root_mean_squared_error: 1.5349 - mean_squared_logarithmic_error: 0.2077
Epoch 93/100
25/25 [==============================] - 0s 6ms/step - loss: 2.4980 - mse: 2.498

25/25 [==============================] - 0s 6ms/step - loss: 2.6493 - mse: 2.6493 - mae: 0.7257 - root_mean_squared_error: 1.6277 - mean_squared_logarithmic_error: 0.2102
Epoch 75/100
25/25 [==============================] - 0s 6ms/step - loss: 2.6075 - mse: 2.6075 - mae: 0.7347 - root_mean_squared_error: 1.6148 - mean_squared_logarithmic_error: 0.2177
Epoch 76/100
25/25 [==============================] - 0s 6ms/step - loss: 2.6950 - mse: 2.6950 - mae: 0.7228 - root_mean_squared_error: 1.6417 - mean_squared_logarithmic_error: 0.2103
Epoch 77/100
25/25 [==============================] - 0s 6ms/step - loss: 2.9429 - mse: 2.9429 - mae: 0.7676 - root_mean_squared_error: 1.7155 - mean_squared_logarithmic_error: 0.2282
Epoch 78/100
25/25 [==============================] - 0s 6ms/step - loss: 2.9825 - mse: 2.9825 - mae: 0.7783 - root_mean_squared_error: 1.7270 - mean_squared_logarithmic_error: 0.2299
Epoch 79/100
25/25 [==============================] - 0s 6ms/step - loss: 2.7247 - mse: 2.724

25/25 [==============================] - 0s 6ms/step - loss: 2.7945 - mse: 2.7945 - mae: 0.7449 - root_mean_squared_error: 1.6717 - mean_squared_logarithmic_error: 0.2181
Epoch 61/100
25/25 [==============================] - 0s 6ms/step - loss: 2.6195 - mse: 2.6195 - mae: 0.7222 - root_mean_squared_error: 1.6185 - mean_squared_logarithmic_error: 0.2097
Epoch 62/100
25/25 [==============================] - 0s 6ms/step - loss: 2.5580 - mse: 2.5580 - mae: 0.7097 - root_mean_squared_error: 1.5994 - mean_squared_logarithmic_error: 0.2004
Epoch 63/100
25/25 [==============================] - 0s 6ms/step - loss: 3.0741 - mse: 3.0741 - mae: 0.7574 - root_mean_squared_error: 1.7533 - mean_squared_logarithmic_error: 0.2320
Epoch 64/100
25/25 [==============================] - 0s 6ms/step - loss: 3.0101 - mse: 3.0101 - mae: 0.8118 - root_mean_squared_error: 1.7350 - mean_squared_logarithmic_error: 0.2513
Epoch 65/100
25/25 [==============================] - 0s 6ms/step - loss: 2.7516 - mse: 2.751

25/25 [==============================] - 0s 6ms/step - loss: 2.8498 - mse: 2.8498 - mae: 0.7445 - root_mean_squared_error: 1.6881 - mean_squared_logarithmic_error: 0.2144
Epoch 47/100
25/25 [==============================] - 0s 6ms/step - loss: 2.9384 - mse: 2.9384 - mae: 0.7810 - root_mean_squared_error: 1.7142 - mean_squared_logarithmic_error: 0.2342
Epoch 48/100
25/25 [==============================] - 0s 6ms/step - loss: 2.8958 - mse: 2.8958 - mae: 0.7758 - root_mean_squared_error: 1.7017 - mean_squared_logarithmic_error: 0.2339
Epoch 49/100
25/25 [==============================] - 0s 6ms/step - loss: 2.7992 - mse: 2.7992 - mae: 0.7551 - root_mean_squared_error: 1.6731 - mean_squared_logarithmic_error: 0.2214
Epoch 50/100
25/25 [==============================] - 0s 6ms/step - loss: 2.8729 - mse: 2.8729 - mae: 0.7718 - root_mean_squared_error: 1.6950 - mean_squared_logarithmic_error: 0.2260
Epoch 51/100
25/25 [==============================] - 0s 6ms/step - loss: 2.8465 - mse: 2.846

...

25/25 [==============================] - 0s 6ms/step - loss: 4.5540 - mse: 4.5540 - mae: 1.0199 - root_mean_squared_error: 2.1340 - mean_squared_logarithmic_error: 0.3600
Epoch 5/100
25/25 [==============================] - 0s 6ms/step - loss: 4.3802 - mse: 4.3802 - mae: 0.9924 - root_mean_squared_error: 2.0929 - mean_squared_logarithmic_error: 0.3394
Epoch 6/100
25/25 [==============================] - 0s 6ms/step - loss: 4.1473 - mse: 4.1473 - mae: 0.9400 - root_mean_squared_error: 2.0365 - mean_squared_logarithmic_error: 0.3130
Epoch 7/100
25/25 [==============================] - 0s 6ms/step - loss: 3.9946 - mse: 3.9946 - mae: 0.8764 - root_mean_squared_error: 1.9986 - mean_squared_logarithmic_error: 0.2901
Epoch 8/100
25/25 [==============================] - 0s 6ms/step - loss: 3.9086 - mse: 3.9086 - mae: 0.8845 - root_mean_squared_error: 1.9770 - mean_squared_logarithmic_error: 0.2967
Epoch 9/100
25/25 [==============================] - 0s 6ms/step - loss: 3.8755 - mse: 3.8755 - m

25/25 [==============================] - 0s 6ms/step - loss: 2.5295 - mse: 2.5295 - mae: 0.7255 - root_mean_squared_error: 1.5904 - mean_squared_logarithmic_error: 0.2070
Epoch 93/100
25/25 [==============================] - 0s 6ms/step - loss: 2.5090 - mse: 2.5090 - mae: 0.7140 - root_mean_squared_error: 1.5840 - mean_squared_logarithmic_error: 0.1987
Epoch 94/100
25/25 [==============================] - 0s 6ms/step - loss: 2.4488 - mse: 2.4488 - mae: 0.7097 - root_mean_squared_error: 1.5649 - mean_squared_logarithmic_error: 0.2010
Epoch 95/100
25/25 [==============================] - 0s 6ms/step - loss: 2.5038 - mse: 2.5038 - mae: 0.7170 - root_mean_squared_error: 1.5823 - mean_squared_logarithmic_error: 0.2043
Epoch 96/100
25/25 [==============================] - 0s 6ms/step - loss: 2.5291 - mse: 2.5291 - mae: 0.7028 - root_mean_squared_error: 1.5903 - mean_squared_logarithmic_error: 0.1965
Epoch 97/100
25/25 [==============================] - 0s 6ms/step - loss: 2.9369 - mse: 2.936

25/25 [==============================] - 0s 6ms/step - loss: 2.6167 - mse: 2.6167 - mae: 0.7472 - root_mean_squared_error: 1.6176 - mean_squared_logarithmic_error: 0.2162
Epoch 79/100
25/25 [==============================] - 0s 6ms/step - loss: 2.6815 - mse: 2.6815 - mae: 0.7680 - root_mean_squared_error: 1.6375 - mean_squared_logarithmic_error: 0.2238
Epoch 80/100
25/25 [==============================] - 0s 6ms/step - loss: 2.6749 - mse: 2.6749 - mae: 0.7570 - root_mean_squared_error: 1.6355 - mean_squared_logarithmic_error: 0.2210
Epoch 81/100
25/25 [==============================] - 0s 6ms/step - loss: 2.6187 - mse: 2.6187 - mae: 0.7472 - root_mean_squared_error: 1.6183 - mean_squared_logarithmic_error: 0.2122
Epoch 82/100
25/25 [==============================] - 0s 6ms/step - loss: 2.5235 - mse: 2.5235 - mae: 0.7501 - root_mean_squared_error: 1.5886 - mean_squared_logarithmic_error: 0.2192
Epoch 83/100
25/25 [==============================] - 0s 6ms/step - loss: 2.6556 - mse: 2.655

25/25 [==============================] - 0s 6ms/step - loss: 2.6548 - mse: 2.6548 - mae: 0.7457 - root_mean_squared_error: 1.6293 - mean_squared_logarithmic_error: 0.2120
Epoch 65/100
25/25 [==============================] - 0s 6ms/step - loss: 2.7121 - mse: 2.7121 - mae: 0.7439 - root_mean_squared_error: 1.6468 - mean_squared_logarithmic_error: 0.2136
Epoch 66/100
25/25 [==============================] - 0s 6ms/step - loss: 2.6186 - mse: 2.6186 - mae: 0.7478 - root_mean_squared_error: 1.6182 - mean_squared_logarithmic_error: 0.2144
Epoch 67/100
25/25 [==============================] - 0s 6ms/step - loss: 2.6157 - mse: 2.6157 - mae: 0.7541 - root_mean_squared_error: 1.6173 - mean_squared_logarithmic_error: 0.2174
Epoch 68/100
25/25 [==============================] - 0s 6ms/step - loss: 2.6333 - mse: 2.6333 - mae: 0.7368 - root_mean_squared_error: 1.6227 - mean_squared_logarithmic_error: 0.2117
Epoch 69/100
25/25 [==============================] - 0s 6ms/step - loss: 2.7731 - mse: 2.773

25/25 [==============================] - 0s 6ms/step - loss: 2.9246 - mse: 2.9246 - mae: 0.7674 - root_mean_squared_error: 1.7102 - mean_squared_logarithmic_error: 0.2288
Epoch 51/100
25/25 [==============================] - 0s 6ms/step - loss: 2.9976 - mse: 2.9976 - mae: 0.7733 - root_mean_squared_error: 1.7314 - mean_squared_logarithmic_error: 0.2245
Epoch 52/100
25/25 [==============================] - 0s 6ms/step - loss: 2.8535 - mse: 2.8535 - mae: 0.7880 - root_mean_squared_error: 1.6892 - mean_squared_logarithmic_error: 0.2316
Epoch 53/100
25/25 [==============================] - 0s 6ms/step - loss: 2.7067 - mse: 2.7067 - mae: 0.7431 - root_mean_squared_error: 1.6452 - mean_squared_logarithmic_error: 0.2154
Epoch 54/100
25/25 [==============================] - 0s 6ms/step - loss: 2.7499 - mse: 2.7499 - mae: 0.7444 - root_mean_squared_error: 1.6583 - mean_squared_logarithmic_error: 0.2157
Epoch 55/100
25/25 [==============================] - 0s 6ms/step - loss: 2.8333 - mse: 2.833

In [22]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/propv85/'

In [23]:
total_training_time

[22.004990577697754,
 21.848665714263916,
 21.93734121322632,
 21.83377456665039,
 22.141991138458252,
 22.314481735229492,
 22.10231328010559,
 22.265422344207764,
 22.03033995628357,
 22.325111150741577]

In [24]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 7.3601477225621545


In [25]:
total_training_loss_history

[{'loss': [5.2022705078125,
   4.729618549346924,
   4.613786697387695,
   4.497284412384033,
   4.295729637145996,
   4.0906548500061035,
   3.8871512413024902,
   3.75875186920166,
   3.6832802295684814,
   3.639071464538574,
   3.5315749645233154,
   3.5651955604553223,
   3.4580724239349365,
   3.372147798538208,
   3.3846611976623535,
   3.3271281719207764,
   3.310502529144287,
   3.3563790321350098,
   3.3274874687194824,
   3.2043330669403076,
   3.2043046951293945,
   3.1982808113098145,
   3.1754331588745117,
   3.1796534061431885,
   3.15299129486084,
   3.2006211280822754,
   3.1881608963012695,
   3.195458173751831,
   3.122128963470459,
   3.0294907093048096,
   3.0464460849761963,
   2.8961260318756104,
   2.8334457874298096,
   3.024132490158081,
   3.0694351196289062,
   3.1739468574523926,
   3.255976915359497,
   2.9608311653137207,
   2.9568660259246826,
   2.922900915145874,
   2.9975290298461914,
   3.00964093208313,
   2.966169595718384,
   2.8148043155670166,
  

In [26]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(10):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 0.8319872617721558
avg mae of training: 0.24018710056940715


In [27]:
total_scores

[[3.0372939109802246,
  3.0372939109802246,
  0.8371924161911011,
  1.7427834272384644,
  0.2733244001865387],
 [2.873776435852051,
  2.873776435852051,
  0.8077849745750427,
  1.6952216625213623,
  0.2620066702365875],
 [2.653724193572998,
  2.653724193572998,
  0.7706341743469238,
  1.6290255784988403,
  0.2509947419166565],
 [2.326166868209839,
  2.326166868209839,
  0.7497791051864624,
  1.5251775979995728,
  0.22886213660240173],
 [3.4453489780426025,
  3.4453489780426025,
  0.980804443359375,
  1.8561651706695557,
  0.3379502594470978],
 [2.852229118347168,
  2.852229118347168,
  0.8242499232292175,
  1.6888543367385864,
  0.2659724950790405],
 [2.7522506713867188,
  2.7522506713867188,
  0.7817718386650085,
  1.6589908599853516,
  0.2412670999765396],
 [2.7589974403381348,
  2.7589974403381348,
  0.8025822043418884,
  1.6610230207443237,
  0.24900250136852264],
 [3.003511667251587,
  3.003511667251587,
  0.8487131595611572,
  1.7330642938613892,
  0.2793237864971161],
 [2.900128

In [1]:
a = [[3.0372939109802246,
  3.0372939109802246,
  0.8371924161911011,
  1.7427834272384644,
  0.2733244001865387],
 [2.873776435852051,
  2.873776435852051,
  0.8077849745750427,
  1.6952216625213623,
  0.2620066702365875],
 [2.653724193572998,
  2.653724193572998,
  0.7706341743469238,
  1.6290255784988403,
  0.2509947419166565],
 [2.326166868209839,
  2.326166868209839,
  0.7497791051864624,
  1.5251775979995728,
  0.22886213660240173],
 [3.4453489780426025,
  3.4453489780426025,
  0.980804443359375,
  1.8561651706695557,
  0.3379502594470978],
 [2.852229118347168,
  2.852229118347168,
  0.8242499232292175,
  1.6888543367385864,
  0.2659724950790405],
 [2.7522506713867188,
  2.7522506713867188,
  0.7817718386650085,
  1.6589908599853516,
  0.2412670999765396],
 [2.7589974403381348,
  2.7589974403381348,
  0.8025822043418884,
  1.6610230207443237,
  0.24900250136852264],
 [3.003511667251587,
  3.003511667251587,
  0.8487131595611572,
  1.7330642938613892,
  0.2793237864971161],
 [2.9001288414001465,
  2.9001288414001465,
  0.819977343082428,
  1.7029764652252197,
  0.2673496901988983]]

In [2]:
test_mae = []
for i in range(10):
    test_mae.append(a[i][2])

In [3]:
test_mae

[0.8371924161911011,
 0.8077849745750427,
 0.7706341743469238,
 0.7497791051864624,
 0.980804443359375,
 0.8242499232292175,
 0.7817718386650085,
 0.8025822043418884,
 0.8487131595611572,
 0.819977343082428]

In [31]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(10):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 0.9534476041793823
avg mae: 0.2741163194179535
avg are: 0.093184694647789


In [32]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 2.5102014541625977
median testing mse: 2.8630027770996094


In [33]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.7226124405860901
median testing mae: 0.8138811588287354


In [34]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.27618730068206787


In [35]:
test_ARE

[0.27356133,
 0.2742312,
 0.27758387,
 0.2921254,
 0.27479076,
 0.27382377,
 0.30305848,
 0.287527,
 0.2515952,
 0.28724384]